In [1]:
# importing packages
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from numpy.linalg import cond, matrix_rank, norm
from scipy.linalg import inv, solve, det, eig, lu, eigvals
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm

In [2]:
#importing data
BL = pd.read_csv('Data/BankLend.csv')
consumption = pd.read_csv('Data/consumption.csv')
gdp_exp = pd.read_csv('Data/gdp.csv')
housing_starts = pd.read_csv('Data/HousingStarts.csv')
ex_rate = pd.read_csv('Data/CEER_MONTHLY_NOMINAL-sd-1990-01-01.csv') #Canadian Effective Exchange Rate - monthly nominal
SP_tbills = pd.read_csv('Data/s&p, tbills.csv')
house_prices = pd.read_csv('Data/OECD Data.csv')

In [3]:
# BANK LENDING

# Summing over rows of 3 to get quarterly data
def sum_quarter(group):
    return pd.Series({'ReferencePeriod': group['ReferencePeriod'].iloc[0],
                      'Total, funds advanced, residential mortgages, insured 7': group['Total, funds advanced, residential mortgages, insured 7'].sum(),
                      'Total, funds advanced, residential mortgages, uninsured 7': group['Total, funds advanced, residential mortgages, uninsured 7'].sum(),
                      'Total, funds advanced, variable rate mortgages': group['Total, funds advanced, variable rate mortgages'].sum(),
                      'Total, funds advanced for non-mortgage loans, consumer credit': group['Total, funds advanced for non-mortgage loans, consumer credit'].sum(),
                      'Total, funds advanced, business loans 8': group['Total, funds advanced, business loans 8'].sum()})

# Group by sets of 3 rows and average using the custom function
BankLend = BL.groupby(BL.index // 3).apply(sum_quarter)
BankLend['ReferencePeriod'] = pd.to_datetime(BankLend['ReferencePeriod'])
BankLend['Quarters'] = BankLend['ReferencePeriod'].dt.to_period("Q").astype(str)

# cleaning up the data
BankLend['BankLending'] = BankLend['Total, funds advanced, residential mortgages, insured 7'] + BankLend['Total, funds advanced, residential mortgages, uninsured 7'] +  BankLend['Total, funds advanced, variable rate mortgages'] + BankLend['Total, funds advanced for non-mortgage loans, consumer credit'] + BankLend['Total, funds advanced, business loans 8'] + BankLend['Total, funds advanced, business loans 8']
BankLend1 = BankLend[['Quarters','BankLending']]
BankLend1.head()

,Quarters,BankLending
0,2013Q1,382924.0
1,2013Q2,493784.0
2,2013Q3,483018.0
3,2013Q4,503383.0
4,2014Q1,420139.0


In [4]:
# GDP 

# keeping specific columns
gdp = gdp_exp[['Quarters', 'Gross domestic product at market prices']]

gdp.head()

,Quarters,Gross domestic product at market prices
0,2000Q1,1514676
1,2000Q2,1532503
2,2000Q3,1548212
3,2000Q4,1551274
4,2001Q1,1560006


In [5]:
# EXCHANGE RATE

# keeping one column
filtered_ex_rate = ex_rate[['date','CEER_BROADNM']]
filtered_ceer = filtered_ex_rate.reset_index(drop=True)

# Averaging over rows of 3 to get quarterly data
def average_quarter(group):
    return pd.Series({'date': group['date'].iloc[0],
                      'CEER_BROADNM': group['CEER_BROADNM'].mean()})

# Group by sets of 3 rows and average using the custom function
ceer = filtered_ceer.groupby(filtered_ceer.index // 3).apply(average_quarter).reset_index(drop=True)

# converting date to datetime format
ceer['date'] = pd.to_datetime(ceer['date'])
ceer['Quarters'] = ceer['date'].dt.to_period("Q").astype(str)

ceer = ceer.drop(['date'], axis=1)
ceer.head()

,CEER_BROADNM,Quarters
0,98.053333,1999Q1
1,101.366667,1999Q2
2,100.096667,1999Q3
3,100.486667,1999Q4
4,102.433333,2000Q1


In [6]:
# CONSUMPTION, HOUSE PRICES, HOUSING STARTS

print(consumption.head())
print(house_prices.head())
print(housing_starts.head())

  Quarters  Household final consumption expenditure
0   2000Q1                                   143884
1   2000Q2                                   145903
2   2000Q3                                   148957
3   2000Q4                                   150284
4   2001Q1                                   151594
  Quarters  House_Prices
0   2001Q1     40.844646
1   2001Q2     41.200980
2   2001Q3     41.875943
3   2001Q4     42.620664
4   2002Q1     43.340875
  Quarters  HousingStarts
0   2016Q1          38175
1   2016Q2          53286
2   2016Q3          54899
3   2016Q4          51555
4   2017Q1          42862


In [7]:
# FINANCIAL INDICATORS: S&P 500, T-BILLS

# Averaging over rows of 3 to get quarterly data
def average_quarter(group):
    return pd.Series({'nummonth': group['nummonth'].iloc[0],
                      'sp': group['sp'].mean(), 't_bills': group['t_bills'].mean()})

# Group by sets of 3 rows and average using the custom function
fin_ind = SP_tbills.groupby(SP_tbills.index // 3).apply(average_quarter).reset_index(drop=True)
fin_ind.rename(columns={'nummonth': 'Quarters'}, inplace=True)
fin_ind.head()

,Quarters,sp,t_bills
0,2013Q1,36782.440000,0.953333
1,2013Q2,36089.766667,1.010000
2,2013Q3,37059.500000,0.990000
3,2013Q4,39750.513333,0.916667
4,2014Q1,41900.943333,0.876667


In [8]:
merge1 = pd.merge(gdp, consumption, on = 'Quarters')
merge2 = pd.merge(merge1, BankLend1, on = 'Quarters')
merge3 = pd.merge(merge2, ceer, on = 'Quarters')
merge4 = pd.merge(merge3, house_prices, on = 'Quarters')
merge5 = pd.merge(merge4, housing_starts, on = 'Quarters')
final_merge = pd.merge(merge5, fin_ind, on = 'Quarters')
final_merge.rename(columns={'Gross domestic product at market prices': 'GDP', 'Household final consumption expenditure': 'CON', 'BankLending': 'BL',
                         'CEER_BROADNM':'FX', 'House_Prices': 'HP', 'HousingStarts': 'HI', 'sp':'OMX', 't_bills':'IR'}, inplace=True)

In [9]:
# Reorder the columns in line with paper
final_df = final_merge[['GDP', 'CON', 'HI', 'HP', 'OMX', 'BL', 'FX', 'IR']]
final_df

,GDP,CON,HI,HP,OMX,BL,FX,IR
0,2071732,285296,38175,105.128205,41244.963333,495823.0,111.046667,0.460000
1,2061412,286760,53286,108.601056,44654.566667,579799.0,117.166667,0.523333
2,2082805,289033,54899,112.826169,46931.136667,586583.0,116.170000,0.516667
3,2094490,292579,51555,115.803167,48612.290000,575781.0,115.810000,0.493333
4,2120843,296967,42862,119.193062,50214.126667,539456.0,117.380000,0.486667
5,2143166,301062,55085,123.993213,50342.013333,639753.0,113.800000,0.563333
6,2143839,303541,61174,125.624057,50567.540000,613865.0,120.043333,0.810000
7,2154716,307583,60642,126.195324,53493.773333,618084.0,118.810000,0.950000
8,2179765,311034,44174,127.509427,52151.660000,579897.0,117.430000,1.143333
9,2196768,314245,57726,127.709276,53897.006667,688764.0,116.433333,1.246667


In [10]:
# Specify the desired column order
desired_order = ['B', 'C', 'A']

# Create a new DataFrame with columns in the desired order
df_reordered = df[desired_order]

NameError: name 'df' is not defined

In [ ]:
final_df['Quarters'] = pd.to_datetime(final_df['Quarters'])
final_df.head() 

,Quarters,Gross domestic product at market prices,Household final consumption expenditure,BankLending,CEER_BROADNM,House_Prices,HousingStarts,sp,t_bills
0,2016-01-01,"2,071,732",285296,495823.0,111.046667,105.128205,"38,175",41244.963333,0.460000
1,2016-04-01,"2,061,412",286760,579799.0,117.166667,108.601056,"53,286",44654.566667,0.523333
2,2016-07-01,"2,082,805",289033,586583.0,116.170000,112.826169,"54,899",46931.136667,0.516667
3,2016-10-01,"2,094,490",292579,575781.0,115.810000,115.803167,"51,555",48612.290000,0.493333
4,2017-01-01,"2,120,843",296967,539456.0,117.380000,119.193062,"42,862",50214.126667,0.486667


In [ ]:
from statsmodels.tsa.vector_ar.svar_model import SVAR

# Assuming your data is in a DataFrame named 'final_df'
# If not, replace 'final_df' with your actual DataFrame

# Define SVAR Model
svar_model = SVAR(final_df[['Gross domestic product at market prices', 'Household final consumption expenditure', 'BankLending', 'CEER_BROADNM', 'House_Prices', 'HousingStarts', 'sp','t_bills']], svar_type='B', A=None, B=None, dates=final_df['Quarters'])

# Impose Short-Term Restrictions for SVAR
B_matrix = ([
    [1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0]])

# Impose Short-Term Restrictions
svar_model.svar_restrict(B_matrix)

# Estimate Model
results = svar_model.fit(maxlags=4)  

# Print Identified Coefficients
identified_results = results.coefs
print(identified_results)

KeyError: "None of [Index(['Gross domestic product at market prices',\n       'Household final consumption expenditure', 'BankLending',\n       'CEER_BROADNM', 'House_Prices', 'HousingStarts', 'sp', 't_bills'],\n      dtype='object')] are in the [columns]"